In [1]:
import os
import json
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
from collections import defaultdict, OrderedDict
import shutil
from tqdm import tqdm

### Json에 있는 Segmentation Annotation 추출 및 line coord 교체

In [2]:
json_dir = os.path.join("/opt/ml/team_github/utils/crosswalk_filtering_closed(12_12).json")

with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)
    images = datas["images"]
    annotations = datas["annotations"]

img_df = pd.DataFrame(images)
annos_df = pd.DataFrame(annotations)

In [3]:
import os
file_list = os.listdir('/opt/ml/data/segmentation/images/all')
len(file_list)

1895

In [4]:
new_img_df = img_df[img_df['file_name'].isin(file_list)].copy()
len(new_img_df)
new_img_df

,license,url,file_name,height,width,data_captured,id
0,0,None,MP_SEL_SUR_000033.jpg,1080,1920,None,0
1,0,None,MP_SEL_SUR_000066.jpg,1080,1920,None,1
2,0,None,MP_SEL_SUR_000084.jpg,1080,1920,None,2
3,0,None,MP_SEL_SUR_000110.jpg,1080,1920,None,3
4,0,None,MP_SEL_SUR_000119.jpg,1080,1920,None,4
...,...,...,...,...,...,...,...
1890,0,None,MP_SEL_SUR_047091.jpg,1080,1920,None,1890
1891,0,None,MP_SEL_SUR_047094.jpg,1080,1920,None,1891
1892,0,None,MP_SEL_SUR_047115.jpg,1080,1920,None,1892
1893,0,None,MP_SEL_SUR_047118.jpg,1080,1920,None,1893


In [5]:
new_img_df.to_csv("seg_filtering_ImageInfo.csv")

In [6]:
new_annos_df = annos_df[annos_df['image_id'].isin(new_img_df['id'])].copy()
len(new_annos_df)
new_annos_df

,id,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,0,3,[],0,"[[1, 795, 356, 799, 704, 875, 704, 1019, 723, ...",0
1,1,0,3,[],0,"[[408, 0, 414, 22, 409, 38, 390, 49, 357, 52, ...",0
2,2,0,3,[],0,"[[955, 0, 955, 51, 975, 55, 991, 53, 993, 30, ...",0
3,3,0,3,[],0,"[[850, 1080, 811, 1079, 807, 1037, 819, 1016, ...",0
4,4,0,1,[],0,"[[1579, 16, 1362, 19, 1351, 27, 1302, 39, 1321...",0
...,...,...,...,...,...,...,...
12542,1,1894,1,[],0,"[[1920, 38, 1124, 28, 1142, 61, 1169, 98, 1192...",0
12543,2,1894,2,[],0,"[[1691, 1080, 1635, 966, 1333, 446, 1279, 326,...",0
12544,3,1894,2,[],0,"[[1108, 0, 1123, 28, 1207, 28, 1203, 20, 1920,...",0
12545,4,1894,2,[],0,"[[1920, 264, 1475, 265, 1603, 415, 1920, 412, ...",0


In [7]:
new_annos_df = new_annos_df.reset_index()
new_annos_df['id'] = new_annos_df.index
new_annos_df.drop(columns='index', inplace=True)
new_annos_df

,id,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,0,3,[],0,"[[1, 795, 356, 799, 704, 875, 704, 1019, 723, ...",0
1,1,0,3,[],0,"[[408, 0, 414, 22, 409, 38, 390, 49, 357, 52, ...",0
2,2,0,3,[],0,"[[955, 0, 955, 51, 975, 55, 991, 53, 993, 30, ...",0
3,3,0,3,[],0,"[[850, 1080, 811, 1079, 807, 1037, 819, 1016, ...",0
4,4,0,1,[],0,"[[1579, 16, 1362, 19, 1351, 27, 1302, 39, 1321...",0
...,...,...,...,...,...,...,...
12542,12542,1894,1,[],0,"[[1920, 38, 1124, 28, 1142, 61, 1169, 98, 1192...",0
12543,12543,1894,2,[],0,"[[1691, 1080, 1635, 966, 1333, 446, 1279, 326,...",0
12544,12544,1894,2,[],0,"[[1108, 0, 1123, 28, 1207, 28, 1203, 20, 1920,...",0
12545,12545,1894,2,[],0,"[[1920, 264, 1475, 265, 1603, 415, 1920, 412, ...",0


In [8]:
new_dict = dict()

for idx, id in enumerate(new_img_df['id'].tolist()):
    new_dict[id] = idx

In [9]:
new_img_df['id'].replace(new_dict, inplace=True)
new_img_df

,license,url,file_name,height,width,data_captured,id
0,0,None,MP_SEL_SUR_000033.jpg,1080,1920,None,0
1,0,None,MP_SEL_SUR_000066.jpg,1080,1920,None,1
2,0,None,MP_SEL_SUR_000084.jpg,1080,1920,None,2
3,0,None,MP_SEL_SUR_000110.jpg,1080,1920,None,3
4,0,None,MP_SEL_SUR_000119.jpg,1080,1920,None,4
...,...,...,...,...,...,...,...
1890,0,None,MP_SEL_SUR_047091.jpg,1080,1920,None,1890
1891,0,None,MP_SEL_SUR_047094.jpg,1080,1920,None,1891
1892,0,None,MP_SEL_SUR_047115.jpg,1080,1920,None,1892
1893,0,None,MP_SEL_SUR_047118.jpg,1080,1920,None,1893


In [10]:
new_annos_df['image_id'] = new_annos_df['image_id'].replace(new_dict)
new_annos_df

,id,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,0,3,[],0,"[[1, 795, 356, 799, 704, 875, 704, 1019, 723, ...",0
1,1,0,3,[],0,"[[408, 0, 414, 22, 409, 38, 390, 49, 357, 52, ...",0
2,2,0,3,[],0,"[[955, 0, 955, 51, 975, 55, 991, 53, 993, 30, ...",0
3,3,0,3,[],0,"[[850, 1080, 811, 1079, 807, 1037, 819, 1016, ...",0
4,4,0,1,[],0,"[[1579, 16, 1362, 19, 1351, 27, 1302, 39, 1321...",0
...,...,...,...,...,...,...,...
12542,12542,1894,1,[],0,"[[1920, 38, 1124, 28, 1142, 61, 1169, 98, 1192...",0
12543,12543,1894,2,[],0,"[[1691, 1080, 1635, 966, 1333, 446, 1279, 326,...",0
12544,12544,1894,2,[],0,"[[1108, 0, 1123, 28, 1207, 28, 1203, 20, 1920,...",0
12545,12545,1894,2,[],0,"[[1920, 264, 1475, 265, 1603, 415, 1920, 412, ...",0


In [11]:
dirpath = '/opt/ml/data/segmentation/images/all'
annpath = '/opt/ml/data/segmentation/annotations/all'

In [12]:
each_area = []
image_see = []
new_seg = {}

print(len(new_annos_df["image_id"]))

for idx, each_record in tqdm(enumerate(new_annos_df.to_dict("record"))):
    seg_datas = each_record["segmentation"]
    h = new_img_df[new_img_df["id"] == each_record["image_id"]]["height"].item() # image h
    w = new_img_df[new_img_df["id"] == each_record["image_id"]]["width"].item()

    for index, target in enumerate(seg_datas):
        coord = []
        for seg_idx in range(0, len(target), 2):
            x, y = target[seg_idx], target[seg_idx+1]
            if x >= w:
                x = w-1
            if y >= h:
                y = h-1
            coord.extend([x, y])
        new_seg.update({idx: [coord]})

/opt/conda/envs/final/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


12547


12547it [00:12, 1006.51it/s]


In [15]:
new_annos_df = new_annos_df.drop('segmentation', axis=1)
s_new_seg = pd.Series(new_seg)
new_annos_df.insert(5, "segmentation", s_new_seg) # insert new_series
new_annos_df

,id,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,0,3,[],0,"[[1, 795, 356, 799, 704, 875, 704, 1019, 723, ...",0
1,1,0,3,[],0,"[[408, 0, 414, 22, 409, 38, 390, 49, 357, 52, ...",0
2,2,0,3,[],0,"[[955, 0, 955, 51, 975, 55, 991, 53, 993, 30, ...",0
3,3,0,3,[],0,"[[850, 1079, 811, 1079, 807, 1037, 819, 1016, ...",0
4,4,0,1,[],0,"[[1579, 16, 1362, 19, 1351, 27, 1302, 39, 1321...",0
...,...,...,...,...,...,...,...
12542,12542,1894,1,[],0,"[[1919, 38, 1124, 28, 1142, 61, 1169, 98, 1192...",0
12543,12543,1894,2,[],0,"[[1691, 1079, 1635, 966, 1333, 446, 1279, 326,...",0
12544,12544,1894,2,[],0,"[[1108, 0, 1123, 28, 1207, 28, 1203, 20, 1919,...",0
12545,12545,1894,2,[],0,"[[1919, 264, 1475, 265, 1603, 415, 1919, 412, ...",0


In [16]:
datas["images"] = new_img_df.to_dict('records')
datas["annotations"] = new_annos_df.to_dict('records')

In [17]:
with open("crosswalk_filtering_poly_closed_noise_filtering(12_12).json", "w") as json_file:
    json.dump(datas, json_file, indent=4)

In [18]:
new1_img_df = new_img_df.copy()
new1_img_df['size'] = None
new1_img_df['ratio'] = None
new1_img_df['size_category'] = None
new1_img_df

,license,url,file_name,height,width,data_captured,id,size,ratio,size_category
0,0,None,MP_SEL_SUR_000033.jpg,1080,1920,None,0,None,None,None
1,0,None,MP_SEL_SUR_000066.jpg,1080,1920,None,1,None,None,None
2,0,None,MP_SEL_SUR_000084.jpg,1080,1920,None,2,None,None,None
3,0,None,MP_SEL_SUR_000110.jpg,1080,1920,None,3,None,None,None
4,0,None,MP_SEL_SUR_000119.jpg,1080,1920,None,4,None,None,None
...,...,...,...,...,...,...,...,...,...,...
1890,0,None,MP_SEL_SUR_047091.jpg,1080,1920,None,1890,None,None,None
1891,0,None,MP_SEL_SUR_047094.jpg,1080,1920,None,1891,None,None,None
1892,0,None,MP_SEL_SUR_047115.jpg,1080,1920,None,1892,None,None,None
1893,0,None,MP_SEL_SUR_047118.jpg,1080,1920,None,1893,None,None,None


In [19]:
dirpath = '/opt/ml/data/segmentation/images/all'
annpath = '/opt/ml/data/segmentation/annotations/all'

In [20]:
Size_Category = ["Small", "Normal", "Large"]

In [21]:
# imageName = "MP_SEL_SUR_000033.jpg"

for imageName in tqdm(new1_img_df['file_name'].values):
    # print(img_df[img_df.file_name==imageName]['id'].item())

    image_path = os.path.join(dirpath, imageName)
    image =cv2.imread(image_path)
    image =cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    ann_data = annos_df[annos_df.image_id==img_df[img_df.file_name==imageName]['id'].item()]['segmentation'].values

    all_coord = []
    fill_image = []
    mask_image = np.zeros([image.shape[0], image.shape[1]])

    for each_ann in ann_data:
        target = each_ann[0]
        coord = []
        for seg_idx in range(0, len(target), 2):
            x, y = target[seg_idx], target[seg_idx+1]
            coord.append([x, y])

        fill_image = cv2.fillPoly(mask_image, [np.array(coord)], 255)
        np_coord = [np.array(coord)]
        np_coord
        image = cv2.polylines(image, np_coord, False, 255, 10)

    # plt.figure(figsize=(20,16))
    # plt.imshow(image)
    # print()
    size = len(np.argwhere(fill_image==255))
    area_ratio = (size / fill_image.size) * 100
    if area_ratio < 8:
        Category = Size_Category[0]
    elif area_ratio < 60:
        Category = Size_Category[1]
    else:
        Category = Size_Category[2]

    new1_img_df.loc[new1_img_df['file_name']==imageName, "size"] = size
    new1_img_df.loc[new1_img_df['file_name']==imageName, "ratio"] = area_ratio
    new1_img_df.loc[new1_img_df['file_name']==imageName, "size_category"] = Category
    # plt.figure(figsize=(20,16))
    # plt.imshow(fill_image)

    # output_path = os.path.join(annpath, imageName.split('.')[0] + ".png")
    # cv2.imwrite(output_path, image)
new1_img_df

100%|██████████| 1895/1895 [02:42<00:00, 11.66it/s]


,license,url,file_name,height,width,data_captured,id,size,ratio,size_category
0,0,None,MP_SEL_SUR_000033.jpg,1080,1920,None,0,1980698,95.5198,Large
1,0,None,MP_SEL_SUR_000066.jpg,1080,1920,None,1,2070486,99.8498,Large
2,0,None,MP_SEL_SUR_000084.jpg,1080,1920,None,2,1573411,75.8782,Large
3,0,None,MP_SEL_SUR_000110.jpg,1080,1920,None,3,1859410,89.6706,Large
4,0,None,MP_SEL_SUR_000119.jpg,1080,1920,None,4,2063585,99.517,Large
...,...,...,...,...,...,...,...,...,...,...
1890,0,None,MP_SEL_SUR_047091.jpg,1080,1920,None,1890,2007845,96.8289,Large
1891,0,None,MP_SEL_SUR_047094.jpg,1080,1920,None,1891,2067370,99.6996,Large
1892,0,None,MP_SEL_SUR_047115.jpg,1080,1920,None,1892,1835819,88.5329,Large
1893,0,None,MP_SEL_SUR_047118.jpg,1080,1920,None,1893,2065646,99.6164,Large


In [ ]:
# imageName = "MP_SEL_SUR_000033.jpg"

    # print(img_df[img_df.file_name==imageName]['id'].item())

image_path = os.path.join(dirpath, imageName)
image =cv2.imread(image_path)
image =cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

ann_data = annos_df[annos_df.image_id==img_df[img_df.file_name==imageName]['id'].item()]['segmentation'].values

all_coord = []
fill_image = []
mask_image = np.zeros([image.shape[0], image.shape[1]])

for each_ann in ann_data:
    target = each_ann[0]
    coord = []
    for seg_idx in range(0, len(target), 2):
        x, y = target[seg_idx], target[seg_idx+1]
        coord.append([x, y])

    fill_image = cv2.fillPoly(mask_image, [np.array(coord)], 255)
    np_coord = [np.array(coord)]
    np_coord
    image = cv2.polylines(image, np_coord, False, 255, 10)

# plt.figure(figsize=(20,16))
# plt.imshow(image)
# print()
size = len(np.argwhere(fill_image==255))
area_ratio = (size / fill_image.size) * 100
if area_ratio < 8:
    Category = Size_Category[0]
elif area_ratio < 60:
    Category = Size_Category[1]
else:
    Category = Size_Category[2]
    # plt.imshow(fill_image)

    # output_path = os.path.join(annpath, imageName.split('.')[0] + ".png")
    # cv2.imwrite(output_path, image)
new1_img_df

In [59]:
new1_img_df.to_csv('image_info_new.csv')

In [15]:
def draw_line(seg_data, mask_image):
    coord = []
    for seg_idx in range(0, len(seg_data), 2):
        x, y = seg_data[seg_idx], seg_data[seg_idx+1]
        if x >= 1920:
            x = 1919
        if y >= 1080:
            y = 1079
        coord.append([x, y])
    
    line_image = cv2.polylines(mask_image, [np.array(coord)], False, 1, 1)
    
    return line_image, np.argwhere(line_image==1)

In [16]:
each_area = []
image_see = []
new_seg = {}

for idx in tqdm(range(len(set(new_annos_df["image_id"])))):
    seg_datas = new_annos_df[new_annos_df['image_id']==idx]["segmentation"]
    h = new_img_df[new_img_df["id"] == idx]["height"].values[0] # image h
    w = new_img_df[new_img_df["id"] == idx]["width"].values[0]

    max_area_ratio = 0
    for index, seg_data in seg_datas.items():
        mask_image = np.zeros([h+1, w+1])
        new_image, new_coord = draw_line(seg_data[0], mask_image)
        
        new_seg.update({index: [list(new_coord.flatten().tolist())]})

100%|██████████| 1896/1896 [00:31<00:00, 60.89it/s]


In [17]:
new_annos_df = new_annos_df.drop('segmentation', axis=1)
new_annos_df

,id,image_id,category_id,bbox,area,iscrowd
0,0,0,1,[],0,0
1,1,0,1,[],0,0
2,2,1,1,[],0,0
3,3,2,1,[],0,0
4,4,3,1,[],0,0
...,...,...,...,...,...,...
2486,2486,1891,1,[],0,0
2487,2487,1892,1,[],0,0
2488,2488,1893,1,[],0,0
2489,2489,1894,1,[],0,0


In [18]:
s_new_seg = pd.Series(new_seg)
new_annos_df.insert(5, "segmentation", s_new_seg) # insert new_series

In [19]:
new_annos_df

,id,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,0,1,[],0,"[[16, 1543, 16, 1544, 16, 1545, 16, 1546, 16, ...",0
1,1,0,1,[],0,"[[46, 1243, 46, 1244, 46, 1245, 46, 1246, 46, ...",0
2,2,1,1,[],0,"[[0, 169, 0, 170, 0, 171, 0, 172, 0, 173, 0, 1...",0
3,3,2,1,[],0,"[[0, 765, 0, 766, 1, 765, 1, 767, 1, 768, 1, 7...",0
4,4,3,1,[],0,"[[69, 768, 69, 769, 69, 770, 69, 771, 69, 772,...",0
...,...,...,...,...,...,...,...
2486,2486,1891,1,[],0,"[[0, 579, 0, 580, 0, 581, 0, 582, 0, 583, 0, 5...",0
2487,2487,1892,1,[],0,"[[0, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 5, 0, 6, 0,...",0
2488,2488,1893,1,[],0,"[[0, 545, 0, 546, 0, 547, 0, 548, 0, 549, 0, 5...",0
2489,2489,1894,1,[],0,"[[0, 577, 0, 578, 0, 579, 0, 580, 0, 581, 0, 5...",0


In [20]:
datas["images"] = new_img_df.to_dict('records')
datas["annotations"] = new_annos_df.to_dict('records')

In [21]:
with open("crosswalk_filtering_seg_noise_filtering.json", "w") as json_file:
    json.dump(datas, json_file, indent=4)

In [25]:
new_img_df.to_csv('image_info.csv')
new_annos_df.to_csv('anno_info.csv')